# Simulate DeFi LP

Jupyter Notebook to simulate liquidity pools (LP) in Unisap v3. Pulls last 60 days of transaction for LP using a Dune Analytics query. Query pulls data for ETH-Stable pairs and Stable-Stable pairs (Only consiers DAI USDC and USDT). Builds a markov chain to forecast the next 7 days of tansactions for each pair.

In [1]:
# Setup
import DuneData
import mcTrainer

### Get LP Data

In [2]:
# Query URL: https://dune.com/queries/2257465
# Query ID
qry_id = 2257465

dex_data = DuneData.DexData(2257465)

TypeError: DexData.get_dune_data() takes 1 positional argument but 2 were given

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import tqdm

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.to(DEVICE)

    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])
        return x

class MarkovChain:
    def __init__(self, rnn, data):
        self.model = rnn
        self.data = data

    def prepare_data(self, data, window_size):
        x, y = [], []
        for i in range(len(data) - window_size):
            x.append(data[i:i + window_size])
            y.append(data[i + window_size])
        return torch.tensor(x).float(), torch.tensor(y).float()
                    
    def train(self, window_size, epochs, batch_size, learning_rate, show_progress=True):
        x, y = self.prepare_data(self.data, window_size)
        dataset = TensorDataset(x, y)
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        criterion = nn.L1Loss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        self.model.train()

        total_steps = len(dataloader) * epochs
        progress_bar = tqdm.tqdm(total=total_steps, desc="Training Progress") if show_progress else None

        for epoch in range(epochs):
            for inputs, targets in dataloader:
                inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
                inputs = inputs.unsqueeze(-1)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, targets.unsqueeze(-1))
                loss.backward()
                optimizer.step()

                if show_progress:
                    progress_bar.update(1)

        if show_progress:
            progress_bar.close()

    def forecast(self, data, window_size):
        self.model.eval()
        with torch.no_grad():
            inputs = torch.tensor(data[-window_size:]).float().to(self.device)
            inputs = inputs.unsqueeze(0).unsqueeze(-1)
            output = self.model(inputs)
            return output.item()


In [ ]:
# Set the hyperparameters
input_size = 1
hidden_size = 50
output_size = 1
lr = 0.001
epochs = 1000
batch_size = 32
window_size = 7


# Prepare the data
data = dune_data['ethereum']['DAI-USDC']['trades'].values.tolist()

# Instantiate the RNN model and the Markov chain
rnn= RNN(input_size, hidden_size, output_size)
mc = MarkovChain(rnn, data)
mc.train(window_size, epochs, batch_size, lr)



In [ ]:
data = time_sereies['ethereum']['DAI-USDC']
x = mcTrainer.train_model(data, 185, 50, 1000)
x.score(plot=True)